In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

In [67]:
total = pd.read_csv("../CSV/train_data.csv")
test = pd.read_csv("../CSV/pred_test.csv")

print("total shape : {}, test shape : {}".format(total.shape, test.shape))

total shape : (368088, 12), test shape : (15120, 12)


In [68]:
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

In [69]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

# "year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"
features = ["month", "hour", "구분_int", "기온(°C)"]
# features = ["year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"]
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

X = total[features]
y = total["공급량"]

In [70]:
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)
print(test.shape)

(306768, 4) (306768,)
(61320, 4) (61320,)
(15120, 12)


In [71]:
model = Sequential()
model.add(Dense(units = 16, input_dim = 4, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(16, activation = "relu"))
model.add(Dense(1))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 16)                80        
_________________________________________________________________
dense_19 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_20 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_21 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_22 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 17        
Total params: 5,361
Trainable params: 5,361
Non-trainable params: 0
____________________________________________________

In [72]:
model.compile(loss = "mean_squared_error", 
              optimizer = "rmsprop", 
              metrics = ["accuracy"])

early_stopping = EarlyStopping(patience = 30)

hist = model.fit(train_x, train_y, 
                 epochs = 20, 
                 batch_size = 10, 
                 validation_data = (val_x, val_y), 
                 callbacks = [early_stopping])

Epoch 1/20
30677/30677 [==============================] - 21s 673us/step - loss: 347600.4062 - accuracy: 0.0000e+00 - val_loss: 55123.0938 - val_accuracy: 0.0000e+00
Epoch 2/20
30677/30677 [==============================] - 23s 735us/step - loss: 38447.0781 - accuracy: 0.0000e+00 - val_loss: 36076.4883 - val_accuracy: 0.0000e+00
Epoch 3/20
30677/30677 [==============================] - 23s 755us/step - loss: 30445.8984 - accuracy: 0.0000e+00 - val_loss: 31689.4961 - val_accuracy: 0.0000e+00
Epoch 4/20
30677/30677 [==============================] - 24s 769us/step - loss: 28591.1465 - accuracy: 0.0000e+00 - val_loss: 45862.5625 - val_accuracy: 0.0000e+00
Epoch 5/20
30677/30677 [==============================] - 22s 723us/step - loss: 27615.5117 - accuracy: 0.0000e+00 - val_loss: 34749.0156 - val_accuracy: 0.0000e+00
Epoch 6/20
30677/30677 [==============================] - 25s 804us/step - loss: 27055.6270 - accuracy: 0.0000e+00 - val_loss: 29357.1016 - val_accuracy: 0.0000e+00
Epoch 7/2

In [73]:
model.evaluate(val_x, val_y)

1917/1917 [==============================] - 2s 730us/step - loss: 43852.1055 - accuracy: 0.0000e+00


[43852.10546875, 0.0]

In [74]:
pred = model.predict(val_x)

true_y = pd.DataFrame({"y" : val_y})
pred_y = pd.DataFrame(pred)

true_y.reset_index(drop = True, inplace = True)
true_y.reset_index(inplace = True)
pred_y.reset_index(inplace = True)

print("NMAE : {:.6f}".format(nmae(true_y, pred_y)))

NMAE : 0.638012


In [62]:
pred_y

,index,0
0,0,2077.645264
1,1,1815.723145
2,2,1752.837646
3,3,1801.679199
4,4,1919.278687
...,...,...
61315,61315,657.350647
61316,61316,656.396667
61317,61317,650.483521
61318,61318,642.748535


In [63]:
true_y

,index,y
0,0,1765.008
1,1,1679.186
2,2,1610.885
3,3,1604.123
4,4,1711.506
...,...,...
61315,61315,681.033
61316,61316,669.961
61317,61317,657.941
61318,61318,610.953


In [64]:
submission = pd.read_csv('../CSV/가스공급량 수요예측 모델개발 data/sample_submission.csv')

In [65]:
test_x = test[features]
preds = model.predict(test_x)
submission['공급량'] = preds
submission.to_csv("../CSV/Submission/17_4_features_DeepLearning_model.csv", index = False)